In [ ]:
#@title Mô phỏng Bảo mật Tin nhắn Âm thanh (với Ghi âm & Nghe lại)

# ==============================================================================
# PHẦN 1: CÀI ĐẶT VÀ CẤU HÌNH
# ==============================================================================
print("⏳ Bước 1: Cài đặt các thư viện cần thiết...")
!pip install flask pyngrok cryptography -q
print("✅ Cài đặt hoàn tất.")

print("🔑 Bước 2: Cấu hình token ngrok...")
!ngrok authtoken 2x78kyYgoItDVATusfDXfO8EB92_7BsHuMDhvAYST2gRrDws2
print("✅ Token đã được cấu hình.")

# ==============================================================================
# PHẦN 2: MÃ NGUỒN ỨNG DỤNG (BACKEND + FRONTEND)
# ==============================================================================
import os
import base64
import json
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
from cryptography.hazmat.primitives.asymmetric import rsa, padding as rsa_padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

app = Flask(__name__)

# --- Giao diện (Frontend) với trình phát Audio ---
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Mô phỏng Bảo mật Tin nhắn Âm thanh</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif; line-height: 1.6; padding: 20px; background-color: #f4f7f9; color: #333; }
        .container { max-width: 1200px; margin: auto; background: white; padding: 25px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); }
        h1, h2 { color: #0056b3; border-bottom: 2px solid #eef; padding-bottom: 10px; }
        .grid { display: grid; grid-template-columns: 1fr 1fr; gap: 30px; }
        .box { padding: 20px; border: 1px solid #ddd; border-radius: 5px; background-color: #fafafa; }
        textarea { width: 100%; height: 150px; padding: 10px; border: 1px solid #ccc; border-radius: 4px; font-family: monospace; font-size: 13px; box-sizing: border-box; }
        button { background-color: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 6px; cursor: pointer; font-size: 16px; margin-top: 10px; transition: background-color 0.3s; }
        button:hover { background-color: #0056b3; }
        button:disabled { background-color: #ccc; cursor: not-allowed; }
        #stop-recording-btn { background-color: #dc3545; }
        #stop-recording-btn:hover { background-color: #c82333; }
        pre { background-color: #eef; padding: 15px; border-radius: 4px; white-space: pre-wrap; word-wrap: break-word; font-size: 13px; max-height: 300px; overflow-y: auto; }
        label { font-weight: bold; margin-bottom: 5px; display: block; }
        .full-width { grid-column: 1 / -1; }
        #recording-status { margin-top: 15px; font-weight: bold; color: #17a2b8; }
        /* THÊM MỚI: Style cho trình phát audio */
        #audio-playback-container { margin-top: 20px; display: none; }
        audio { width: 100%; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Mô phỏng Truyền tin Âm thanh An toàn (AES-256 & RSA-2048)</h1>
        <div class="box full-width">
            <h2>Bước 1: Tạo khóa</h2>
            <p>Nhấn nút để tạo các cặp khóa RSA cho Alice (Người gửi) và Bob (Người nhận).</p>
            <button id="generate-keys-btn">Tạo Khóa cho cả Alice và Bob</button>
        </div>
        <div class="grid">
            <div class="box">
                <h2>Phía Người gửi (Alice)</h2>
                <label for="alice-public-key">Khóa công khai của Alice:</label>
                <textarea id="alice-public-key" readonly></textarea>
                <label for="alice-private-key">Khóa riêng của Alice:</label>
                <textarea id="alice-private-key" readonly></textarea>
                <hr>
                <label for="bob-public-key-input">Khóa công khai của Bob (đã tự điền):</label>
                <textarea id="bob-public-key-input"></textarea>
                <hr>
                <label>Ghi âm Tin nhắn thoại:</label>
                <div>
                    <button id="start-recording-btn" disabled>Bắt đầu Ghi âm</button>
                    <button id="stop-recording-btn" disabled>Dừng Ghi âm</button>
                </div>
                <div id="recording-status">Trạng thái: Sẵn sàng. (Vui lòng tạo khóa trước)</div>
                <div id="audio-playback-container">
                    <label>Nghe lại bản ghi âm đã gửi:</label>
                    <audio id="audio-playback" controls></audio>
                </div>
            </div>
            <div class="box">
                <h2>Phía Người nhận (Bob)</h2>
                <label for="bob-public-key">Khóa công khai của Bob:</label>
                <textarea id="bob-public-key" readonly></textarea>
                <label for="bob-private-key">Khóa riêng của Bob:</label>
                <textarea id="bob-private-key" readonly></textarea>
            </div>
        </div>
        <div class="box full-width">
            <h2>Kết quả Xử lý</h2>
            <h3>Gói tin được gửi đi (JSON):</h3>
            <pre id="packet-output">Gói tin sẽ xuất hiện ở đây...</pre>
            <h3>Nhật ký xử lý (Log):</h3>
            <pre id="log-output">Nhật ký các bước sẽ xuất hiện ở đây...</pre>
        </div>
    </div>
    <script>
        // DOM Elements
        const generateBtn = document.getElementById('generate-keys-btn');
        const startRecordingBtn = document.getElementById('start-recording-btn');
        const stopRecordingBtn = document.getElementById('stop-recording-btn');
        const statusDiv = document.getElementById('recording-status');
        const audioPlayback = document.getElementById('audio-playback'); // THÊM MỚI
        const audioPlaybackContainer = document.getElementById('audio-playback-container'); // THÊM MỚI

        // Key TextAreas
        const alicePubKeyTA = document.getElementById('alice-public-key');
        const alicePrivKeyTA = document.getElementById('alice-private-key');
        const bobPubKeyTA = document.getElementById('bob-public-key');
        const bobPrivKeyTA = document.getElementById('bob-private-key');
        const bobPubKeyInputTA = document.getElementById('bob-public-key-input');

        // Output Elements
        const packetOutput = document.getElementById('packet-output');
        const logOutput = document.getElementById('log-output');

        // MediaRecorder variables
        let mediaRecorder;
        let audioChunks = [];

        generateBtn.addEventListener('click', async () => {
            logOutput.textContent = 'Đang tạo khóa...';
            try {
                const response = await fetch('/generate-all-keys', { method: 'POST' });
                const data = await response.json();
                alicePubKeyTA.value = data.alice_public_key;
                alicePrivKeyTA.value = data.alice_private_key;
                bobPubKeyTA.value = data.bob_public_key;
                bobPrivKeyTA.value = data.bob_private_key;
                bobPubKeyInputTA.value = data.bob_public_key;
                logOutput.textContent = 'Đã tạo khóa thành công! Sẵn sàng để ghi âm.';
                startRecordingBtn.disabled = false;
            } catch (error) {
                logOutput.textContent = `Lỗi khi tạo khóa: ${error.message}.`;
            }
        });

        startRecordingBtn.addEventListener('click', async () => {
            try {
                const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                mediaRecorder = new MediaRecorder(stream);
                audioChunks = [];
                audioPlaybackContainer.style.display = 'none'; // Ẩn trình phát cũ

                mediaRecorder.ondataavailable = event => {
                    audioChunks.push(event.data);
                };

                mediaRecorder.onstop = async () => {
                    const audioBlob = new Blob(audioChunks, { type: 'audio/webm' });

                    // THÊM MỚI: Hiển thị trình phát audio để nghe lại
                    const audioUrl = URL.createObjectURL(audioBlob);
                    audioPlayback.src = audioUrl;
                    audioPlaybackContainer.style.display = 'block';

                    const formData = new FormData();
                    formData.append('audio_data', audioBlob, 'recording.webm');
                    formData.append('alice_private_key', alicePrivKeyTA.value);
                    formData.append('alice_public_key', alicePubKeyTA.value);
                    formData.append('bob_public_key', bobPubKeyInputTA.value);
                    formData.append('bob_private_key', bobPrivKeyTA.value);

                    logOutput.textContent = 'Đã dừng ghi âm. Đang gửi dữ liệu để mã hóa và xử lý...';
                    packetOutput.textContent = 'Đang xử lý...';

                    const response = await fetch('/process-message', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json();

                    if (result.success) {
                        packetOutput.textContent = JSON.stringify(result.packet, null, 2);
                    } else {
                        packetOutput.textContent = 'Có lỗi xảy ra.';
                    }
                    logOutput.textContent = result.log;
                };

                mediaRecorder.start();
                statusDiv.textContent = "🔴 Đang ghi âm...";
                startRecordingBtn.disabled = true;
                stopRecordingBtn.disabled = false;

            } catch (err) {
                statusDiv.textContent = `Lỗi: ${err.message}. Bạn đã cho phép truy cập micro chưa?`;
            }
        });

        stopRecordingBtn.addEventListener('click', () => {
            mediaRecorder.stop();
            statusDiv.textContent = "Trạng thái: Sẵn sàng ghi âm.";
            startRecordingBtn.disabled = false;
            stopRecordingBtn.disabled = true;
        });
    </script>
</body>
</html>
"""

# --- LOGIC MÁY CHỦ (BACKEND) - Không có thay đổi ở phần này ---
def generate_key_pair():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    pem_private = private_key.private_bytes(encoding=serialization.Encoding.PEM, format=serialization.PrivateFormat.PKCS8, encryption_algorithm=serialization.NoEncryption()).decode('utf-8')
    pem_public = public_key.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo).decode('utf-8')
    return pem_private, pem_public

def deserialize_keys(pem_private_key, pem_public_key):
    private_key = serialization.load_pem_private_key(pem_private_key.encode('utf-8'), password=None) if pem_private_key else None
    public_key = serialization.load_pem_public_key(pem_public_key.encode('utf-8')) if pem_public_key else None
    return private_key, public_key

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/generate-all-keys', methods=['POST'])
def generate_all_keys_api():
    alice_private, alice_public = generate_key_pair()
    bob_private, bob_public = generate_key_pair()
    return jsonify({
        "alice_public_key": alice_public, "alice_private_key": alice_private,
        "bob_public_key": bob_public, "bob_private_key": bob_private
    })

@app.route('/process-message', methods=['POST'])
def process_message_api():
    log = []
    try:
        audio_file = request.files.get('audio_data')
        if not audio_file:
            raise ValueError("Không nhận được dữ liệu âm thanh.")

        alice_pem_private = request.form.get('alice_private_key')
        alice_pem_public = request.form.get('alice_public_key')
        bob_pem_public = request.form.get('bob_public_key')
        bob_pem_private = request.form.get('bob_private_key')

        audio_message_bytes = audio_file.read()

        log.append("--- Bắt đầu phía Người gửi (Alice) ---")
        alice_private_key, _ = deserialize_keys(alice_pem_private, None)
        _, bob_public_key = deserialize_keys(None, bob_pem_public)
        aes_key = os.urandom(32)
        log.append("1. Alice: Tạo khóa phiên AES-256 ngẫu nhiên.")
        encrypted_aes_key = bob_public_key.encrypt(aes_key, rsa_padding.OAEP(mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None))
        log.append("2. Alice: Mã hóa khóa AES bằng khóa công khai của Bob.")
        iv = os.urandom(16)
        cipher_aes = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
        padder = PKCS7(algorithms.AES.block_size).padder()
        padded_data = padder.update(audio_message_bytes) + padder.finalize()
        ciphertext = cipher_aes.encryptor().update(padded_data)
        log.append("3. Alice: Mã hóa tin nhắn thoại bằng AES-256 CBC.")
        data_to_hash = iv + ciphertext
        hasher = hashes.Hash(hashes.SHA256())
        hasher.update(data_to_hash)
        final_hash = hasher.finalize()
        log.append(f"4. Alice: Tạo hash SHA-256.")
        signature = alice_private_key.sign(final_hash, rsa_padding.PSS(mgf=rsa_padding.MGF1(hashes.SHA256()), salt_length=rsa_padding.PSS.MAX_LENGTH), hashes.SHA256())
        log.append("5. Alice: Dùng khóa riêng của mình để ký lên hash.")
        packet = {"encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode('utf-8'), "iv": base64.b64encode(iv).decode('utf-8'), "cipher": base64.b64encode(ciphertext).decode('utf-8'), "hash": final_hash.hex(), "sig": base64.b64encode(signature).decode('utf-8')}
        log.append("6. Alice: Đóng gói và gửi đi.")

        log.append("\n--- Bắt đầu phía Người nhận (Bob) ---")
        _, alice_public_key = deserialize_keys(None, alice_pem_public)
        bob_private_key_obj, _ = deserialize_keys(bob_pem_private, None)
        decrypted_aes_key = bob_private_key_obj.decrypt(base64.b64decode(packet['encrypted_aes_key']), rsa_padding.OAEP(mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None))
        log.append("1. Bob: Dùng khóa riêng giải mã thành công khóa AES.")
        log.append("2. Bob: Bắt đầu kiểm tra chữ ký và tính toàn vẹn...")
        recalculated_hash_obj = hashes.Hash(hashes.SHA256())
        recalculated_hash_obj.update(base64.b64decode(packet['iv']) + base64.b64decode(packet['cipher']))
        recalculated_hash = recalculated_hash_obj.finalize()
        if recalculated_hash.hex() != packet['hash']:
            raise ValueError("Lỗi Toàn vẹn (Integrity Error)! Hash không khớp.")
        log.append("   - Kiểm tra Hash Toàn vẹn: OK.")
        alice_public_key.verify(base64.b64decode(packet['sig']), recalculated_hash, rsa_padding.PSS(mgf=rsa_padding.MGF1(hashes.SHA256()), salt_length=rsa_padding.PSS.MAX_LENGTH), hashes.SHA256())
        log.append("   - Xác thực Chữ ký của Alice: OK.")
        log.append("3. Bob: Mọi thứ hợp lệ. Tiến hành giải mã tin nhắn thoại...")
        decryptor_cipher = Cipher(algorithms.AES(decrypted_aes_key), modes.CBC(base64.b64decode(packet['iv'])))
        unpadder = PKCS7(algorithms.AES.block_size).unpadder()
        decrypted_padded_msg = decryptor_cipher.decryptor().update(base64.b64decode(packet['cipher']))
        decrypted_msg_bytes = unpadder.update(decrypted_padded_msg) + unpadder.finalize()

        log.append(f"   - Giải mã thành công! Nhận được {len(decrypted_msg_bytes)} bytes dữ liệu âm thanh.")
        log.append(f"\n✅ TIN NHẮN THOẠI ĐÃ ĐƯỢC GIẢI MÃ AN TOÀN! (Gửi ACK)")

        return jsonify({"success": True, "packet": packet, "log": "\n".join(log)})
    except Exception as e:
        log.append(f"\n❌ NACK: ĐÃ XẢY RA LỖI: {str(e)}")
        return jsonify({"success": False, "log": "\n".join(log)})

# ==============================================================================
# PHẦN 3: KHỞI CHẠY ỨNG DỤNG
# ==============================================================================
print("🚀 Bước 3: Khởi chạy ứng dụng web...")
public_url = ngrok.connect(5000)
print(f"🌍 Ứng dụng của bạn đang chạy tại URL công khai: {public_url}")
app.run(port=5000)

⏳ Bước 1: Cài đặt các thư viện cần thiết...
✅ Cài đặt hoàn tất.
🔑 Bước 2: Cấu hình token ngrok...
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Token đã được cấu hình.
🚀 Bước 3: Khởi chạy ứng dụng web...
🌍 Ứng dụng của bạn đang chạy tại URL công khai: NgrokTunnel: "https://4997-34-106-71-184.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 12:41:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 12:41:57] "GET /favicon.ico HTTP/1.1" 404 -
